In [1]:
import numpy as np
from six.moves import cPickle as pickle
from scipy import ndimage
from scipy.misc import imresize, imsave

pixels_width = 32
pixels_height = 32
file_end = ".png"
pixels_depth = 255.0

In [2]:
pickleFile = open('svhn.pickle', 'r')
data = pickle.load(pickleFile)
training = data['train']
testing = data['test']
valid = data['valid']
predict = data['predict']

In [3]:
def removeZeros(arr):
    newarr = []
    for index in range(len(arr)):
        if arr[index] > 0:
            newarr.append(arr[index])
    return newarr
  
def getLeftBound(lefts):
    if len(lefts) <= 0:
        return 0
    return int(min(lefts))

def getTopBound(tops):
    if len(tops) <= 0:
        return 0
    return int(min(tops))

def getBottomBound(top, heights):
    return int(max(heights) + top)

def getRightBound(lefts, widths):
    if len(lefts) == 0:
        left = 0
    else:
        left = max(lefts)
    
    if len(widths) == 0:
        width = 0
    else:
        width = np.sum(np.fabs(widths))
        
    return int(left + width)

In [4]:
def manipulateImage(path, dataset, index):
    name = dataset['images'][index]
    imageFile = path + name + file_end
    imageData = np.average(ndimage.imread(imageFile),axis=2)
    lefts = removeZeros(dataset['lefts'][index])
    widths = removeZeros(dataset['widths'][index])
    tops = removeZeros(dataset['tops'][index])
    heights = removeZeros(dataset['heights'][index])
    
    left = getLeftBound(lefts)
    right = getRightBound(lefts, widths)
    top = getTopBound(tops)
    if top < 0: top = 0
    bottom = getBottomBound(top, heights)
    
    cutImage = imageData[top:bottom, left:right]
    if cutImage.size == 0:
        cutImage = imageData
    resizedData = imresize(cutImage, (pixels_height, pixels_width))
    resizedData = (resizedData.astype(float) - pixels_depth / 2) / pixels_depth
    return resizedData

def generateNewImages(path, dataset):
    indices = dataset['images'].shape[0]
    generatedData = np.ndarray(shape=(indices, pixels_height, pixels_width), dtype=np.float32)
    counter = 0
    for index in range(indices):
        image = manipulateImage(path, dataset, index)
        generatedData[index, :, :] = image

    return generatedData

In [5]:
training_X = generateNewImages("train/", training)
testing_X = generateNewImages("test/", testing)
valid_X = generateNewImages("extra/", valid)
predict_X = generateNewImages("extra/", predict)

In [6]:
print(training_X.shape)
print(testing_X.shape)
print(valid_X.shape)
print(predict_X.shape)

(33402, 32, 32)
(13068, 32, 32)
(15000, 32, 32)
(5000, 32, 32)


In [10]:
training_Y = training['labels']
testing_Y = testing['labels']
valid_Y = valid['labels']
predict_Y = predict['labels']

In [8]:
training_Y[0]

array([  1.,   9.,  10.,  10.,  10.,  10.])

In [11]:
def createDataLabelDict(X,Y):
    return {
        'data': X,
        'label': Y
    }

tensorflow_file = 'tensorflow_data.pickle'
try:
    with open(tensorflow_file, 'wb') as f:
        dump = {
            'train': createDataLabelDict(training_X, training_Y),
            'test': createDataLabelDict(testing_X, testing_Y),
            'valid': createDataLabelDict(valid_X, valid_Y),
            'predict': createDataLabelDict(predict_X, predict_Y)
        }
        pickle.dump(dump, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', tensorflow_file, ':', e)